# Synthetic Function with Noise

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


## Define function

$$y = sin(2\pi x) + \epsilon$$

where $\epsilon$ is Gaussian (from normal distribution) noise 

In [3]:
# Define synthetic function
def f(x, epsilon):
    np.sin(2 * np.pi * x) + epsilon

In [4]:
# Define different noise levels from different standard deviation
sd1 = np.random.normal(scale=0.1)
sd2 = np.random.normal(scale=0.25)
sd3 = np.random.normal(scale=1.0)

In [ ]:
x = np.random.uniform(low=-2.0, high=2)